# Argo / GO-SHIP Comparison

This will serve as our working notebook, broken into our discussed working components below. 

## This cell for imports, constants

In [1]:
# data processing
import numpy as np
import pandas as pd
from time import sleep

#data visualization
%matplotlib inline
import matplotlib.pyplot as plt
%matplotlib inline

# API convenience functions
from utilities_NSF_EC2022 import get_data_for_timeRange

import warnings
warnings.filterwarnings('ignore')

# set constants
URL_PREFIX = 'https://argovis-api.colorado.edu'
API_KEY = ''

## This cell for data loading/processing to xarray

In [2]:
# more convenient wrapper for if you want a line just by name
def get_goship_line(line_name, startDate='1900-01-01T00:00:00Z', endDate='2022-05-01T00:00:00Z', dt_tag='365d', url=URL_PREFIX, api_key=API_KEY):
    df = get_data_for_timeRange(startDate, endDate, url_prefix=url+'/profiles?', 
                                source='cchdo_go-ship', woceline=line_name, 
                                myAPIkey=api_key, dt_tag=dt_tag)
    return df

# I think we should have a time independent version of this function as well
# could cheat this function into doing it with something like timedelta=1e5 or something
def get_argo_along_line(time, coords, radius=50, timedelta=30, dt_tag='365d', url=URL_PREFIX, api_key=API_KEY):
    df_all = pd.DataFrame()
    for t, c in zip(time, coords):
        sleep(.2)
        startDate = (pd.Timestamp(t) - pd.Timedelta(timedelta/2)).strftime('%Y-%m-%dT%H:%M:%SZ')
        endDate   = (pd.Timestamp(t) + pd.Timedelta(timedelta/2)).strftime('%Y-%m-%dT%H:%M:%SZ')
        center    = f'{c[0]},{c[1]}'
        df = get_data_for_timeRange(startDate, endDate, url_prefix=url+'/profiles?',
            center=center, radius_km=f'{radius}', source='argo_core', data='pres,temp,psal',
            myAPIkey=api_key, dt_tag=dt_tag, writeFlag=False)
        df_all = df_all.append(df)
    
    return df_all

# get argo data along line A22
a22 = get_goship_line('A22')
coords = [c['coordinates'] for c in a22.geolocation]
time = a22.timestamp.values
argo_a22 = get_argo_along_line(time, coords)

In [ ]:
# process data from full of JSON points to more usable form

def json_to_dataframe(df, keys=['pres', 'temp', 'psal']):
    # for i in range(df.shape[0]):
    data_dict = dict()
    for i in range(1):
        

## This cell for Data Visualization

In [ ]:
from argopy import DataFetcher as ArgoDataFetcher
ArgoSet = ArgoDataFetcher().float([6902746])

ds = ArgoSet.to_xarray()

## This cell for Data Interpolation/Comparison